In [ ]:
import requests, json, base64, urllib.parse

In [ ]:
tenant = ""
clientId = ''
clientSecret = ''
 
domain = "identity.oraclecloud.com"
port = "443"
baseurl = 'https://' + tenant + '.' + domain + ':' + port
configurl =  baseurl + '/.well-known/idcs-configuration'

In [ ]:
config = None
response = requests.get(configurl).text
config = json.loads(response)
tokenurl = config["openid-configuration"]["token_endpoint"]

grant_type = "client_credentials"
scope = "urn:opc:idm:__myscopes__"

basicauthHeader = base64.b64encode(bytes(clientId + ":" + clientSecret, 'utf-8')) 
reqdata = {'grant_type': grant_type, 'scope' : scope}
reqheaders = {'Authorization': 'Basic ' + basicauthHeader.decode("utf-8"), 'content-type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data = reqdata, headers = reqheaders)
accesstoken = response.json()["access_token"]
headers = {'Authorization': 'Bearer ' + accesstoken, 'content-type': 'application/json'}
print(accesstoken)

In [ ]:
def getapproleids(approlename):
    approleurl = baseurl + "/admin/v1/AppRoles"
    filter = "?filter=displayName eq \"" + approlename + "\""
    approlelist = requests.get(approleurl + filter, headers = headers).json()["Resources"]
    
    if(len(approlelist) > 0):
        return approlelist[0]["id"], approlelist[0]["app"]["value"]
    else:
        return None, None

def testbulkpayload(approleid, appid):
    payload = {
        "schemas": [
            "urn:ietf:params:scim:api:messages:2.0:BulkRequest"
          ],
        "Operations": [
              {
                  "method": "POST",
                  "path": "/Users",
                  "bulkId": "ukid_user",
                  "data": {
                    "schemas": [
                          "urn:ietf:params:scim:schemas:core:2.0:User",
                          "urn:ietf:params:scim:schemas:oracle:idcs:extension:user:User", 
                          "urn:ietf:params:scim:schemas:oracle:idcs:extension:userState:User"
                    ],
                    "userName": "bulkuser1",
                    "name": {
                      "givenName": "Bulk",
                      "familyName": "User1",
                      "formatted": "Bulk User1"
                    },
                    "displayName": "Bulk User1",
                    "active": True,
                    "emails": [
                          {
                            "value": "demoidcs+X@gmail.com",
                            "type": "work",
                            "primary": True
                          }
                    ]
                  }
            },
            {
                "method": "POST",
                "path": "/Grants",
                "bulkId": "ukid_grant",
                "data": {
                    "schemas": [
                        "urn:ietf:params:scim:schemas:oracle:idcs:Grant"
                      ],
                     "app": {
                        "value": appid
                      },
                      "entitlement": {
                        "attributeName": "appRoles",
                        "attributeValue": approleid
                      },
                      "grantMechanism": "ADMINISTRATOR_TO_USER",
                      "grantee": {
                        "value": "bulkId:ukid_user",
                        "type": "User"
                      }
                }
            }
      ]
    }
    return payload

In [ ]:
appolename = 'JaaS_Administrator'
approleid,appid = getapproleids(appolename)


bulkurl = baseurl + "/admin/v1/Bulk"   
bulkpayload = testbulkpayload(approleid, appid)
response = requests.post(bulkurl, data = json.dumps(bulkpayload), headers = headers).json()
print(response)